In [1]:
import pandas as pd
import os
import json
import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import csv
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from chemdataextractor import Document

In [2]:
doi_index = []
text = []

In [3]:
pathacs = '/Users/juliachotoo/ScrapyArticles/scrapedjson/acs2/'

for filename in os.listdir(pathacs):
    with open(pathacs + filename, 'r'):
        resultacs = json.load(open(pathacs + filename, mode='r'))
        doi_index.append(resultacs['doi'])
        text.append(resultacs['text'])

In [4]:
pathspr = '/Users/juliachotoo/ScrapyArticles/scrapedjson/spr2/'

for filename in os.listdir(pathspr):
    with open(pathspr + filename, 'r'):
        resultspr = json.load(open(pathspr + filename, mode='r'))
        doi_index.append(resultspr['doi'])
        text.append(resultspr['text'])

In [5]:
pathlabels = '/Users/juliachotoo/ScrapyArticles/datalabels.csv'
labels = []

with open(pathlabels, 'r') as csvfile:
    resultlabels = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in resultlabels:
        labels.append(', '.join(row))

labels[0] = labels[0].replace('\ufeff', '')
print(labels)    


['0', '1', '0', '1', '0', '0', '2', '0', '2', '0', '0', '0', '0', '0', '0', '1', '0', '0', '2', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '2', '2', '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '2', '1', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '2', '0', '1', '0', '0', '0', '2', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '2', '0', '0', '0', '0', '2', '0', '1', '1', '2', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '1', '0', '1', '0', '0', '2', '0', '0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0',

In [104]:
labels.count('1')

43

In [6]:
df = pd.DataFrame({'data': text, 'labels': labels})
print(df.loc[0:9])

                                                data labels
0  10.1021/ic501413r Metal–organic frameworks (MO...      0
1  10.1021/jp102881e  Top of PageAbstractIntroduc...      1
2  10.1021/acs.analchem.5b00391 Metal–organic fra...      0
3  10.1021/ie071645b  Top of PageAbstract1. Intro...      1
4  10.1021/ic502478u Metal–organic frameworks (MO...      0
5  10.1021/cm4034319 Top of PageAbstractIntroduct...      0
6  10.1021/acs.langmuir.5b04185 In the original a...      2
7  10.1021/ie500310c Top of PageAbstractIntroduct...      0
8  10.1021/ie051056a  Top of PageAbstractIntroduc...      2
9  10.1021/ja5022014 Top of PageAbstractIntroduct...      0


In [7]:
X = df.data
y = df[df.columns[1]]
print(X.shape)
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(267,)
(267,)
(200,)
(67,)
(200,)
(67,)


In [94]:
def stemming_tokenizer(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in word_tokenize(text)]
vect = CountVectorizer(tokenizer=stemming_tokenizer, stop_words=stopwords.words('english'))
vect.fit(X_train)
X_train_dtm = vect.fit_transform(X_train)


X_test_dtm = vect.transform(X_test)
X_test_dtm

<67x39481 sparse matrix of type '<class 'numpy.int64'>'
	with 53757 stored elements in Compressed Sparse Row format>

In [28]:
#tf_transformer = TfidfTransformer()
#X_train_tf = tf_transformer.fit_transform(X_train_dtm)
#X_test_tf = tf_transformer.transform(X_test_dtm)

In [84]:
nb = MultinomialNB()

In [95]:
%time nb.fit(X_train_dtm, y_train)

CPU times: user 5.72 ms, sys: 1.7 ms, total: 7.41 ms
Wall time: 5.42 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [96]:
y_pred_class = nb.predict(X_test_dtm)

In [97]:
metrics.accuracy_score(y_test, y_pred_class)

0.80597014925373134

In [98]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[50,  2,  0],
       [ 3,  4,  0],
       [ 5,  3,  0]])

In [99]:
logreg = LogisticRegression()

In [100]:
%time logreg.fit(X_train_dtm, y_train)

CPU times: user 794 ms, sys: 18.3 ms, total: 813 ms
Wall time: 214 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [101]:
y_pred_class2 = logreg.predict(X_test_dtm)

In [102]:
metrics.accuracy_score(y_test, y_pred_class2)

0.83582089552238803

In [103]:
metrics.confusion_matrix(y_test, y_pred_class2)

array([[51,  1,  0],
       [ 2,  4,  1],
       [ 6,  1,  1]])

In [115]:
from sklearn.svm import SVC


In [150]:
svcm = SVC(kernel='linear', gamma=1, class_weight= {'2':100})
%time svcm.fit(X_train_dtm, y_train)

CPU times: user 321 ms, sys: 1.9 ms, total: 323 ms
Wall time: 322 ms


SVC(C=1.0, cache_size=200, class_weight={'2': 100}, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [151]:
y_pred_class3 = svcm.predict(X_test_dtm)

In [152]:
metrics.accuracy_score(y_test, y_pred_class3)

0.79104477611940294

In [153]:
metrics.confusion_matrix(y_test, y_pred_class3)

array([[49,  3,  0],
       [ 3,  4,  0],
       [ 7,  1,  0]])